<a href="https://colab.research.google.com/github/cfong32/netflix-prize/blob/main/try_sparktorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install sparktorch pyspark

In [2]:
from sparktorch import serialize_torch_obj, SparkTorch
import torch
import torch.nn as nn
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import SparkSession
from pyspark.ml.pipeline import Pipeline

In [ ]:
spark = SparkSession.builder.appName("examples").master('local[2]').getOrCreate()
df = spark.read.option("inferSchema", "true").option("header", "true").csv('mnist_train.csv').coalesce(2)

In [6]:
df = df.sample(fraction=0.01)

In [7]:
network = nn.Sequential(
    nn.Linear(784, 256),
    nn.ReLU(),
    nn.Linear(256, 256),
    nn.ReLU(),
    nn.Linear(256, 10),
    nn.Softmax(dim=1)
)

# Build the pytorch object
torch_obj = serialize_torch_obj(
    model=network,
    criterion=nn.CrossEntropyLoss(),
    optimizer=torch.optim.Adam,
    lr=0.0001
)

# Setup features
vector_assembler = VectorAssembler(inputCols=df.columns[1:785], outputCol='features')

# Create a SparkTorch Model with torch distributed. Barrier execution is on by default for this mode.
spark_model = SparkTorch(
    inputCol='features',
    labelCol='label',
    predictionCol='predictions',
    torchObj=torch_obj,
    iters=50,
    verbose=1
)

In [8]:
# Can be used in a pipeline and saved.
p = Pipeline(stages=[vector_assembler, spark_model]).fit(df)
p.save('simple_dnn')

In [10]:
vars(p.stages[-1])

{'_input_kwargs': {'inputCol': 'features',
  'predictionCol': 'predictions',
  'modStr': '...\n',
  'useVectorOut': False},
 'uid': 'SparkTorchModel_40897fd41a85',
 '_paramMap': {Param(parent='SparkTorchModel_40897fd41a85', name='inputCol', doc='input column name.'): 'features',
  Param(parent='SparkTorchModel_40897fd41a85', name='predictionCol', doc='prediction column name.'): 'predictions',
  Param(parent='SparkTorchModel_40897fd41a85', name='modStr', doc=''): '...\n',
  Param(parent='SparkTorchModel_40897fd41a85', name='useVectorOut', doc=''): False},
 '_defaultParamMap': {Param(parent='SparkTorchModel_40897fd41a85', name='predictionCol', doc='prediction column name.'): 'predicted',
  Param(parent='SparkTorchModel_40897fd41a85', name='inputCol', doc='input column name.'): 'encoded',
  Param(parent='SparkTorchModel_40897fd41a85', name='modStr', doc=''): '',
  Param(parent='SparkTorchModel_40897fd41a85', name='useVectorOut', doc=''): False},
 '_params': None,
 'inputCol': Param(parent

In [11]:
predictions = p.transform(df)

In [12]:
predictions.show()

+-----+---+---+---+---+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+---+---+---+---+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+---+---+---+---+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+---+---+---+---+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+---+---+---+---+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+---+---+---+---+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+---+---+---+---+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+---+---+---+---+---+---+---+---+---+----+----+----+----+----+----+----+----+